In [1]:
import pandas as pd
import pymysql
from sqlalchemy import create_engine

In [21]:
# Configuração dos parâmetros do banco
host='localhost'
port=3306
database='constructal_automate_results'
user='constructal_automate'
password='constructal_design_2024'

query = f'''
SELECT 
    sp.N_ls, 
    sp.N_ts, 
    sp.k, 
    sp.h_s, 
    sp.t_s, 
    (p.b + sp.N_ls * sp.h_s) AS L_eq, 
    (p.b * sp.t_1 + sp.N_ls * sp.h_s * sp.t_s) AS A_eq,
    ROUND((p.b * sp.t_1 + sp.N_ls * sp.h_s * sp.t_s)/(p.b + sp.N_ls * sp.h_s), 2) AS t_eq,
    ROUND((epb1.n_u * (p.b + sp.N_ls * sp.h_s) / (p.b * sp.t_1 + sp.N_ls * sp.h_s * sp.t_s)),2) as TENSAO_1A,
    ROUND((epb1.n_u * (p.b + sp.N_ls * sp.h_s) / (p.b * sp.t_1 + sp.N_ls * sp.h_s * sp.t_s)) / 186.46, 3) AS NUS_1A,
    ROUND((epb2.n_u /sp.t_1), 2) AS TENSAO_2A,
    ROUND((epb2.n_u /sp.t_1) / 59.68, 3) AS NUS_2A
FROM constructal_automate_results.csg_stiffenedplate AS sp
INNER JOIN constructal_automate_results.cbeb_stiffenedplateanalysis AS spa1 
    ON spa1.stiffened_plate_id = sp.id AND spa1.buckling_load_type_id = 1
INNER JOIN constructal_automate_results.cbeb_elastoplasticbuckling AS epb1 
    ON epb1.stiffened_plate_analysis_id = spa1.id
INNER JOIN constructal_automate_results.cbeb_stiffenedplateanalysis AS spa2 
    ON spa2.stiffened_plate_id = sp.id AND spa2.buckling_load_type_id = 2
INNER JOIN constructal_automate_results.cbeb_elastoplasticbuckling AS epb2  
    ON epb2.stiffened_plate_analysis_id = spa2.id and epb2.id not between 259 and 266
INNER JOIN constructal_automate_results.csg_plate AS p 
    ON p.id = sp.plate_id;
'''

base_path = 'tabelas_dissertacao/'

In [3]:
file_name = 'estudo_comparativo_lima_2016_com_tensao'
csv_name = file_name + ".csv"
latex_name = file_name + ".tex"

In [4]:
def retrieve_data_from_db(host, port, user, password, database, query):
    try:
        # Criar a conexão com o banco de dados
        connection_string = f"mysql+pymysql://{user}:{password}@{host}:{port}/{database}"
        engine = create_engine(connection_string)

        # Executar a query e carregar os dados em um DataFrame
        df = pd.read_sql(query, engine)
        return df
    except Exception as e:
        print(f"Erro: {e}")

In [22]:
df_results = retrieve_data_from_db(host, port, user, password, database, query)

In [23]:
df_results

,N_ls,N_ts,k,h_s,t_s,L_eq,A_eq,t_eq,TENSAO_1A,NUS_1A,TENSAO_2A,NUS_2A
0,0,0,0.000,0.0,0.0,1000.0,14000.0,14.00,186.46,1.000,59.68,1.000
1,2,2,56.200,281.0,5.0,1562.0,12610.0,8.07,31.03,0.166,25.57,0.428
2,2,2,14.100,141.0,10.0,1282.0,12620.0,9.84,352.90,1.893,196.18,3.287
3,2,2,6.333,95.0,15.0,1190.0,12650.0,10.63,325.50,1.746,211.45,3.543
4,2,2,3.550,71.0,20.0,1142.0,12640.0,11.07,312.62,1.677,219.08,3.671
...,...,...,...,...,...,...,...,...,...,...,...,...
124,5,5,1.450,29.0,20.0,1145.0,12700.0,11.09,290.49,1.558,120.23,2.015
125,5,5,0.960,24.0,25.0,1120.0,12800.0,11.43,265.50,1.424,100.37,1.682
126,5,5,0.666,20.0,30.0,1100.0,12800.0,11.64,236.84,1.270,87.20,1.461
127,5,5,0.485,17.0,35.0,1085.0,12775.0,11.77,213.55,1.145,82.18,1.377


In [15]:
def general_formatter(fmt, value):
    return f'{fmt}' % value

def formatter_d(value):
    return '%d' % value

def formatter_float1(value):
    return '%.1f' % value

def formatter_float2(value):
    return '%.2f' % value

def formatter_float3(value):
    return '%.3f' % value

def formatter_float4(value):
    return '%.4f' % value

In [24]:
def generate_latex_from_df(df):
    # Salvar o resultado como um arquivo CSV
    df.to_csv(base_path + csv_name, index=False)

    formatters = ['%d', '%d']

    # Converter o DataFrame para LaTeX
    print(f"Convertendo o arquivo CSV para LaTeX")
    latex_table = df.to_latex(formatters=[formatter_d, formatter_d, formatter_float3, formatter_d, formatter_d, formatter_d, formatter_d, formatter_float2, formatter_float2, formatter_float3, formatter_float2, formatter_float3],
                              decimal=',',
                              index=False)

    # Salvar a tabela LaTeX em um arquivo .tex
    with open(base_path + latex_name, "w") as f:
        f.write(latex_table)

    print("Tabela LaTeX criada com sucesso!")

In [25]:
generate_latex_from_df(df_results)

Convertendo o arquivo CSV para LaTeX
Tabela LaTeX criada com sucesso!
